## Import Libraries

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from faker import Faker
import random
np.random.seed()
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from model import *
from utils import *

## Create Dataset

In [9]:
x = []
palindromes = []
for i in range(1024):
    binary_string = format(i, '010b')
    x.append(binary_string)
    if binary_string == binary_string[::-1]:
        palindromes.append(binary_string)
x = np.array(x)
palindromes = np.array(palindromes)


In [10]:
print(len(palindromes))
palindromes

32


array(['0000000000', '0000110000', '0001001000', '0001111000',
       '0010000100', '0010110100', '0011001100', '0011111100',
       '0100000010', '0100110010', '0101001010', '0101111010',
       '0110000110', '0110110110', '0111001110', '0111111110',
       '1000000001', '1000110001', '1001001001', '1001111001',
       '1010000101', '1010110101', '1011001101', '1011111101',
       '1100000011', '1100110011', '1101001011', '1101111011',
       '1110000111', '1110110111', '1111001111', '1111111111'],
      dtype='<U10')

In [11]:
np.random.shuffle(palindromes)
print(len(palindromes))
palindromes

32


array(['0110110110', '1101001011', '0010110100', '0001001000',
       '0110000110', '1011111101', '0100000010', '1010110101',
       '0000000000', '0011111100', '1001001001', '0011001100',
       '0111001110', '1011001101', '1001111001', '1000110001',
       '1000000001', '0101001010', '1110110111', '0001111000',
       '0111111110', '1110000111', '0010000100', '1101111011',
       '0101111010', '1100110011', '1111001111', '0100110010',
       '0000110000', '1100000011', '1111111111', '1010000101'],
      dtype='<U10')

In [126]:
for i in range(5):
    x = np.concatenate((x, palindromes))
print(len(x))

1184


In [127]:
y = []
for binary_string in x:
    y.append(binary_string == binary_string[::-1])
y = np.array(y)
permutation_index = np.random.permutation(len(x))
x = x[permutation_index]
y = y[permutation_index]

In [128]:
def sigmoid(x):
    return 1 / (1 + np.exp(-1 * x))

def relu(x):
    return np.maximum(0, x)

In [129]:
# a = np.array([1, 2, -3])
# b = np.array([2, 3, 4])
# a*b

array([  2,   6, -12])

In [13]:
LEARNING_RATE = 1e-3
NUM_EPOCHS = 10000

input_size = 10
hidden_layer_size = 20
output_size = 1

weights_ih = np.random.rand(input_size, hidden_layer_size)
weights_ho = np.random.rand(hidden_layer_size, output_size)

In [ ]:
x = np.array([])

In [131]:
# LEARNING_RATE = 1e-7
for epoch in range(NUM_EPOCHS):
    tot_loss = 0
    for i, s in enumerate(x):
        inp = np.reshape(np.array([int(char) for char in s]), (1, input_size))
        hlayer_logits = np.dot(inp, weights_ih)
        hlayer_output = sigmoid(hlayer_logits)

        final_logits = np.dot(hlayer_output, weights_ho)
        final_output = sigmoid(final_logits)

        tot_loss += abs(final_output - y[i])

        output_delta = (y[i] - final_output) * final_output * (1 - final_output)

        weights_ho += LEARNING_RATE * hlayer_output.T.dot(output_delta)
        weights_ih += LEARNING_RATE * inp.T.dot(output_delta.dot(weights_ho.T) * hlayer_output * (1 - hlayer_output))

    if epoch % 100 == 0:
        print(f'Epoch {epoch} : loss {tot_loss}')

Epoch 0 : loss [[991.61380209]]
Epoch 100 : loss [[989.2316454]]
Epoch 200 : loss [[319.53270712]]
Epoch 300 : loss [[319.57421463]]
Epoch 400 : loss [[319.61364199]]
Epoch 500 : loss [[319.6512102]]
Epoch 600 : loss [[319.6870946]]
Epoch 700 : loss [[319.72143354]]
Epoch 800 : loss [[319.75433739]]
Epoch 900 : loss [[319.78589815]]
Epoch 1000 : loss [[319.81620019]]
Epoch 1100 : loss [[319.84533201]]
Epoch 1200 : loss [[319.87339917]]
Epoch 1300 : loss [[319.90053773]]
Epoch 1400 : loss [[319.92692755]]
Epoch 1500 : loss [[319.9528046]]
Epoch 1600 : loss [[319.97847091]]
Epoch 1700 : loss [[320.00430116]]
Epoch 1800 : loss [[320.03074492]]
Epoch 1900 : loss [[320.05832377]]
Epoch 2000 : loss [[320.08762303]]
Epoch 2100 : loss [[320.11927799]]
Epoch 2200 : loss [[320.15395461]]
Epoch 2300 : loss [[320.19232432]]
Epoch 2400 : loss [[320.23503171]]
Epoch 2500 : loss [[320.28265285]]
Epoch 2600 : loss [[320.33564028]]
Epoch 2700 : loss [[320.39424877]]
Epoch 2800 : loss [[320.45843371]]
E

In [139]:
LEARNING_RATE = 1e-1
for epoch in range(NUM_EPOCHS):
    tot_loss = 0
    for i, s in enumerate(x):
        inp = np.reshape(np.array([int(char) for char in s]), (1, input_size))
        hlayer_logits = np.dot(inp, weights_ih)
        hlayer_output = sigmoid(hlayer_logits)

        final_logits = np.dot(hlayer_output, weights_ho)
        final_output = sigmoid(final_logits)

        tot_loss += abs(final_output - y[i])

        output_delta = (y[i] - final_output) * final_output * (1 - final_output)

        weights_ho += LEARNING_RATE * hlayer_output.T.dot(output_delta)
        weights_ih += LEARNING_RATE * inp.T.dot(output_delta.dot(weights_ho.T) * hlayer_output * (1 - hlayer_output))

    if epoch % 100 == 0:
        print(f'Epoch {epoch} : loss {tot_loss}')

Epoch 0 : loss [[0.70983834]]
Epoch 100 : loss [[0.7091767]]
Epoch 200 : loss [[0.70852412]]
Epoch 300 : loss [[0.70787714]]
Epoch 400 : loss [[0.70723423]]
Epoch 500 : loss [[0.70659466]]
Epoch 600 : loss [[0.70595803]]


KeyboardInterrupt: 

In [143]:
x_test1 = '1100110011'
x_test2 = '1100110000'
def test(x):
    inp = np.reshape(np.array([int(char) for char in x]), (1, input_size))
    return sigmoid(np.dot(sigmoid(np.dot(inp, weights_ih)), weights_ho))

print(test(x_test1))
print(test(x_test2))

[[0.99802693]]
[[9.87927878e-05]]


In [144]:
# np.save('weights_ih.npy', weights_ih)
# np.save('weights_ho.npy', weights_ho)

## New implementation

In [1]:
from utils import activation, dataset, losses
from model import *
import numpy as np
# np.random.seed(7)


Dataset = dataset.Palindrome_Dataset(n_bits=10)
model = Palindrome_Model(input_size=10, output_size=1,hidden_layer_sizes=[4],activation='sigmoid')

In [2]:

x,y = Dataset.get_data(shuffle=True, biasing_factor=20)

/home/screa/Desktop/Sem8/CS772/assign1-palindrome/utils/dataset.py:29: UserWarning: you are shuffling a 'flatiter' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(x.flat)


In [3]:
model.set_optimizer(lr=0.001,loss="bce")
model.layers

[{'name': 'layer-1',
  'weights': array([[ 0.53395377,  0.23218934, -0.28231011, -0.59693814],
         [ 0.45579783, -1.23376619,  0.47297493,  0.09048471],
         [ 0.2898087 , -0.40248243, -0.28032975,  0.33549292],
         [-0.01318235,  0.78686707, -0.51578312, -0.16396029],
         [ 0.92861159,  0.07549704,  1.10145881, -0.23826712],
         [ 0.196562  , -0.41292983, -0.29635503, -0.29084583],
         [-0.51092125, -0.02046567, -0.99829916,  0.02753536],
         [ 0.20506253,  0.06761003, -0.24076845,  0.33732247],
         [-0.31232972,  0.45552252, -0.0904001 , -0.1570045 ],
         [ 0.04495314,  0.28331099, -0.31902885,  0.09601966]]),
  'biases': array([-0.93115687,  1.07258205, -0.89303028,  0.03468321]),
  'activation': 'linear'},
 {'name': 'layer-2',
  'weights': array([[-0.26644013],
         [-0.31473152],
         [ 0.25819514],
         [-0.26744164]]),
  'biases': array([-0.43499538]),
  'activation': 'sigmoid'}]

In [4]:
accuracies, losses = model.train(x,y,epochs=100)

Epoch 100/100: 100%|██████████| 1664/1664 [00:00<00:00, 8821.16sample/s, accuracy=59.6, loss=3.94e+3] 


In [13]:
temp_x = x[:48]
temp_y = y[:48]
print(temp_x, temp_y)

[[1 1 0 1 0 0 1 0 1 0]
 [0 1 0 1 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 1 0]
 [0 1 0 0 1 1 0 0 1 1]
 [1 0 1 1 1 1 1 1 0 1]
 [1 1 1 1 1 0 0 1 1 1]
 [1 1 1 1 0 0 0 0 1 1]
 [0 1 0 1 0 0 1 0 1 0]
 [0 1 0 1 1 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 1 1 0 0]
 [0 1 0 1 1 1 1 0 0 0]
 [0 0 1 1 1 1 1 1 0 0]
 [0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 1 0 1 0 1]
 [1 1 1 1 0 0 0 1 0 1]
 [1 1 1 0 1 1 0 1 1 1]
 [0 0 0 0 0 1 1 1 0 1]
 [0 1 0 1 1 1 1 0 1 0]
 [0 1 0 0 0 0 1 0 1 1]
 [0 0 0 1 1 1 1 0 0 0]
 [1 0 0 1 1 0 0 0 1 0]
 [1 0 1 1 0 1 0 0 0 0]
 [0 1 0 1 0 1 0 0 1 1]
 [1 0 0 0 1 1 1 0 0 1]
 [0 1 0 0 0 0 1 1 0 1]
 [0 1 0 1 0 0 1 0 1 0]
 [0 0 1 0 0 1 0 0 0 0]
 [0 1 1 0 1 1 0 1 1 0]
 [0 1 0 0 0 1 1 0 1 0]
 [1 0 0 1 1 1 1 0 0 1]
 [0 0 0 0 0 1 1 1 1 1]
 [0 0 1 0 1 1 1 0 1 0]
 [0 0 0 1 0 0 1 0 0 0]
 [0 1 0 0 1 1 0 0 1 0]
 [0 0 1 1 1 0 1 1 1 1]
 [1 1 1 0 0 0 0 1 1 1]
 [0 0 1 1 0 1 0 0 0 1]
 [0 1 0 0 0 0 0 0 1 0]
 [1 1 0 1 1 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 0 0 1 1 0]
 [0 1 0 1 0 1 1 0 0 0]
 [1 1 0 0 0

#### Resolving Bug

In [15]:
x_palindromes = np.array([np.reshape(np.array([int(char) for char in s]), (10,)) for s in Dataset.palindromes])
y_palindromes = np.array([1]*32)

x_non_palindrome = np.array([np.reshape(np.array([int(char) for char in s]), (10,)) for s in Dataset.non_palindromes])
y_non_palindrome = np.array([0]*len(x_non_palindrome))
temp_x = np.concatenate((temp_x,x_palindromes))
temp_y = np.concatenate((temp_y, y_palindromes))
accuracies, losses = model.train(x_palindromes,y_palindromes,epochs=10)
# print(temp_x, temp_y)

Epoch 10/10: 100%|██████████| 32/32 [00:00<00:00, 6074.57sample/s, accuracy=100, loss=477]


In [6]:
# model.train(x_non_palindrome, y_non_palindrome, epochs=20)

model.set_optimizer(lr=0.005,loss="mse")
accuracies, losses = model.train(temp_x,temp_y,epochs=100)
# model.train(x_palindromes, y_palindromes, epochs=100)

Epoch 100/100: 100%|██████████| 80/80 [00:00<00:00, 5464.18sample/s, accuracy=66.2, loss=7.71]


In [9]:
predictions = model.predict(x_palindromes)
accuracy_metric(predictions, y_palindromes)


100.0

In [10]:
predictions = model.predict(x_non_palindrome)
# print(predictions)
accuracy_metric(predictions, y_non_palindrome)

17.036290322580644

### Batch  Optimization

In [ ]:
from utils import activation, dataset, losses
from model import *
import numpy as np
# np.random.seed(7)

Dataset = dataset.Palindrome_Dataset(n_bits=10)
x,y = Dataset.get_data(shuffle=True, biasing_factor=20)
model = Palindrome_Model(input_size=10, output_size=1,hidden_layer_sizes=[4,4],activation='sigmoid')
model.set_optimizer(lr=0.005,loss="bce")

In [ ]:
temp = np.array([1,1,1,1,1,1,1,1,0,1])
y_temp = [1]

### K-Fold Cross Validation

In [8]:
def k_fold_cross_validation(model,X:np.ndarray, y:np.ndarray, k_folds:int = 4, epochs_per_fold:int = 25):
        fold_size = len(X) // k_folds
        accuracies_fold = []
        accuracies=[]
        losses = []
        for i in range(k_folds):
            start, end = i * fold_size, (i + 1) * fold_size
            X_test_fold, y_test_fold = X[start:end], y[start:end]
            X_train_fold = np.concatenate([X[:start], X[end:]])
            y_train_fold = np.concatenate([y[:start], y[end:]])
            print(f"Training Fold {i+1}\n")
            accuracy, loss = model.train(X_train_fold, y_train_fold, epochs_per_fold)
            accuracies.extend(accuracy)
            losses.extend(loss)
            print("\n"+10*"----"+"\n")
            predictions = model.predict(X_test_fold)
            accuracies_fold.append(accuracy_metric(predictions, y_test_fold))
        return accuracies_fold

In [9]:
accs = k_fold_cross_validation(model, x, y, epochs_per_fold=50)

Training Fold 1



Epoch 50/50: 100%|██████████| 1248/1248 [00:00<00:00, 5973.76sample/s, accuracy=59.5, loss=72.4]



----------------------------------------

Training Fold 2



Epoch 50/50: 100%|██████████| 1248/1248 [00:00<00:00, 5874.39sample/s, accuracy=59.5, loss=72.2]



----------------------------------------

Training Fold 3



Epoch 50/50: 100%|██████████| 1248/1248 [00:00<00:00, 6090.02sample/s, accuracy=60.8, loss=82.3]



----------------------------------------

Training Fold 4



Epoch 50/50: 100%|██████████| 1248/1248 [00:00<00:00, 5977.70sample/s, accuracy=55.6, loss=67.9]


----------------------------------------



In [10]:
np.average(accs)

59.01442307692308

In [11]:
accs

[60.09615384615385, 60.09615384615385, 56.00961538461539, 59.855769230769226]

In [8]:
predictions = model.predict(x)
accuracy_metric(predictions, y)

59.61538461538461

In [9]:
model.layers

[{'name': 'layer-1',
  'weights': array([[ 0.92075584,  0.98020454, -0.17354498,  0.54963896, -0.61341082],
         [-0.3069449 , -0.31859287, -0.7935808 ,  0.24452297, -1.06643863],
         [ 0.32264924, -0.32461446, -0.21445626, -0.47678489, -0.25638755],
         [-0.62537194, -0.19053209, -0.21555031, -0.40445053, -0.1660093 ],
         [-0.07174505,  0.42259881, -0.24471143, -0.01031693, -0.81025356],
         [ 0.69404405,  0.21085697, -0.28454665,  0.52845819, -0.09958278],
         [-0.14040927,  0.51493797,  0.30452068, -0.20036355,  0.52997397],
         [ 0.35928047,  0.55598655,  0.20385861,  0.29601196, -0.03650077],
         [-0.37006127, -0.24986717, -0.1882553 ,  0.28326173,  0.48526483],
         [-0.69906679,  0.24546093, -0.45782992,  0.78897229, -0.65061248]]),
  'biases': array([ 1.03922811, -0.59154367, -0.61716551, -0.83110799,  0.51746456]),
  'activation': 'linear'},
 {'name': 'layer-2',
  'weights': array([[ 0.03572792],
         [ 0.08053775],
         [-0.

In [11]:
count = 0
for i in range(len(y)):
    if predictions[i]==1:
        count+=1
        # print(x[i], y[i], predictions[i])
count

0

In [16]:
count

0

In [17]:
len(x)

1664

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
model = models.Sequential([
    layers.InputLayer(input_shape=(10,)),          # Input layer with size 10
    layers.Dense(10, activation='linear'),          # Hidden layer with size 4 and linear activation
    layers.Dense(1, activation='sigmoid')           # Output layer with size 1 and sigmoid activation
])

# Compile the model
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
model.fit(x, y, epochs=10, batch_size=8)

Epoch 1/10
208/208 [==============================] - 0s 931us/step - loss: 0.6762 - accuracy: 0.5962
Epoch 2/10
208/208 [==============================] - 0s 923us/step - loss: 0.6759 - accuracy: 0.5962
Epoch 3/10
208/208 [==============================] - 0s 864us/step - loss: 0.6761 - accuracy: 0.5962
Epoch 4/10
208/208 [==============================] - 0s 876us/step - loss: 0.6759 - accuracy: 0.5962
Epoch 5/10
208/208 [==============================] - 0s 870us/step - loss: 0.6760 - accuracy: 0.5962
Epoch 6/10
208/208 [==============================] - 0s 860us/step - loss: 0.6756 - accuracy: 0.5962
Epoch 7/10
208/208 [==============================] - 0s 966us/step - loss: 0.6762 - accuracy: 0.5962
Epoch 8/10
208/208 [==============================] - 0s 967us/step - loss: 0.6761 - accuracy: 0.5962
Epoch 9/10
208/208 [==============================] - 0s 1ms/step - loss: 0.6760 - accuracy: 0.5962
Epoch 10/10
208/208 [==============================] - 0s 1ms/step - loss: 0.6760 - 